# Mission

**Prêt à dépenser**, une société financière spécialisée dans les crédits à la consommation pour des individus sans historique de prêt, souhaite développer un outil de "scoring crédit" pour calculer la probabilité qu’un client rembourse son crédit.

L'objectif est de prédire la probabilité de remboursement d'un client et de classifier les demandes en crédit accordé ou refusé en utilisant un algorithme de classification.

La mission implique :

* Mise en œuvre d'une démarche MLOps en utilisant des outils Open Source :
  - MLFlow : tracking d'expérimentations, stockage centralisé.
  - Git : gestion de code.
  - Github :  partage de code.
  - Github Actions : Déploiement de l'API de manière continue.
  - Pytest : tests unitaires.
  - Evidently : Data drift en production.
* Construction du modèle de scoring : 
  - Analyse des features qui contribuent le plus au modèle (feature importance locale et globale).
  - Prise en compte du déséquilibre des classes et du coût métier entre faux négatifs et faux positifs.
  - Optimisation des hyperparamètres
* Mise en production du Modèle de scoring :
  - Déploiement de l'API sur une plateforme Cloud (Azure ou autre)
  - Interface de test en utilisant Streamlit

# Importation des données

In [8]:
# Importation des librairies

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import time

Description des données :

* **application_(train|test)** : les principales données de train et de test avec des informations sur chaque demande de prêt chez Home Credit. Chaque prêt a sa propre ligne et est identifié par la caractéristique SK_ID_CURR. Les données de train sont accompagnées de la caractéristique TARGET indiquant 0 : le prêt a été remboursé ou 1 : le prêt n'a pas été remboursé.
* **bureau** : données concernant les crédits antérieurs du client auprès d'autres institutions financières. Chaque crédit précédent a sa propre ligne dans bureau, mais un prêt existant dans application_(train|test) peut avoir plusieurs crédits précédents.
* **bureau_balance** : données mensuelles concernant les crédits précédents dans bureau. Chaque ligne correspond à un mois de crédit précédent, et un crédit précédent unique peut avoir plusieurs lignes, une pour chaque mois de la durée du crédit.
* **previous_application** : demandes précédentes de prêts a Home Credit des clients qui ont des prêts dans les données de application_(train|test). Chaque prêt actuel dans les données de application_(train|test) peut avoir plusieurs prêts précédents. Chaque demande précédente a une ligne et est identifiée par la caractéristique SK_ID_PREV.
* **POS_CASH_BALANCE** : données mensuelles sur les prêts au point de vente ou en espèces que les clients ont eu avec Home Credit. Chaque ligne correspond à un mois d'un prêt au point de vente ou d'un prêt en espèces précédent, et un seul prêt précédent peut avoir plusieurs lignes.
* **credit_card_balance** : données mensuelles sur les cartes de crédit que les clients ont eu avec Home Credit. Chaque ligne correspond à un mois de solde de carte de crédit, et une seule carte de crédit peut avoir plusieurs lignes.
* **installments_payment** : historique des paiements pour les prêts précédents chez Home Credit. Il y a une ligne pour chaque paiement effectué et une ligne pour chaque paiement manqué.


Ce diagramme montre comment toutes les données sont liées:

![home_credit.png](home_credit.png)

In [10]:
!mlflow ui

^C


Modeles : LightGBM, XGBoost, CatBoost
Metriques :
- ROC AUC, precision, recall, f1, matrice de confusion

# Exploration des données